In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
from opening_hours import OpeningHours
import re

In [3]:
df = pd.read_csv('osm_places_amenity.csv')

C:\Users\Daniel\AppData\Local\Temp\ipykernel_22052\2753785423.py:1: DtypeWarning: Columns (47,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('osm_places_amenity.csv')


In [4]:
hours = df['opening_hours'].astype(str).unique()

In [5]:
df.shape

(14647, 90)

In [6]:
# Split by ;
hours = [hour.strip() for hrs in hours for hour in hrs.split(';')]

In [7]:
# For now let's just look at unique statements
hours = list(set(hours))

In [8]:
print(hours)

['We-Th 16:00-23:00', 'Mo-Th 11:30-15:30,17:00-22:30', 'Mo-Fr 07:00-16:00', 'We 06:30-24:00', 'Su 16:00-02:00', 'Sa-Su 11:00-12:00', 'Mo-Su 11:45-23:00', 'We,Th 16:00-21:00', 'Fr,Sa 11:00-00:30', 'Sa, Su 13:00-04:00', 'Sa-Su 15:00-21:00', 'Fr,Sa 11:00-22:00 "Food Hall"', 'Fr-Sa 11:00-01:45', 'Mo-Sa 10:00-24:00', 'Mo 08:00-18:00', 'Mo-Fr 08:00-22:00', 'Tu-Th 11:00-20:00', 'Mo-Th, Su 11:30-24:00', 'We 06:30-16:00', 'Mo-We 10:30-24:00', 'Tu-Th 11:30-15:00,17:00-22:00', 'Tu-Fr 07:00-18:00', 'Su 12:00-21:00 "Bar" || Mo-Th,Su 11:00-21:00', 'Mo-We, Su 11:30-00:00', 'Su 11:30-15:00,17:30-20:30', 'Mo-Th 06:15-19:00', '04:00-22:00', 'Mo 17:00-01:00', 'Sa-Su 09:30-20:30', 'Su 07:00-19:00', 'Fr 11:00-17:30', 'Mo-Th, Su 10:30-23:00', 'Fr-Sa 09:00-01:00', 'Mo-Fr 10:00-19:45', 'Mo-Fr 09:00-21:00', 'Mo-Th 11:30-16:00, 17:00-22:30', 'We-Th 17:00-23:00', 'Mo-Fr 11:00-15:30', 'Mo-Su 12:15-20:30', 'Tu-Fr 12:00-24:00', 'Su 05:00-19:00', 'Mo, We 13:00-00:00', 'Sa-Su 00:00-24:00', 'We-Sa 08:30-00:00', 'Mo-We

In [9]:
# Change shortenings of some days for the OpeningHours parser to work.
day_map = {
    "Mo":"Mon",
    "Tu":"Tue",
    "We":"Wed",
    "Fr": "Fri"
}

string = 'Mo-Th 12:00-22:00'
for i, _ in enumerate(hours):
    for key, val in day_map.items():
        hours[i] = hours[i].replace(key, val)

In [10]:
# Split by commas
hours = [[hr.strip()  for hr in hrs.split(',')] for hrs in hours]

In [11]:
print(hours)

[['Wed-Th 16:00-23:00'], ['Mon-Th 11:30-15:30', '17:00-22:30'], ['Mon-Fri 07:00-16:00'], ['Wed 06:30-24:00'], ['Su 16:00-02:00'], ['Sa-Su 11:00-12:00'], ['Mon-Su 11:45-23:00'], ['Wed', 'Th 16:00-21:00'], ['Fri', 'Sa 11:00-00:30'], ['Sa', 'Su 13:00-04:00'], ['Sa-Su 15:00-21:00'], ['Fri', 'Sa 11:00-22:00 "Food Hall"'], ['Fri-Sa 11:00-01:45'], ['Mon-Sa 10:00-24:00'], ['Mon 08:00-18:00'], ['Mon-Fri 08:00-22:00'], ['Tue-Th 11:00-20:00'], ['Mon-Th', 'Su 11:30-24:00'], ['Wed 06:30-16:00'], ['Mon-Wed 10:30-24:00'], ['Tue-Th 11:30-15:00', '17:00-22:00'], ['Tue-Fri 07:00-18:00'], ['Su 12:00-21:00 "Bar" || Mon-Th', 'Su 11:00-21:00'], ['Mon-Wed', 'Su 11:30-00:00'], ['Su 11:30-15:00', '17:30-20:30'], ['Mon-Th 06:15-19:00'], ['04:00-22:00'], ['Mon 17:00-01:00'], ['Sa-Su 09:30-20:30'], ['Su 07:00-19:00'], ['Fri 11:00-17:30'], ['Mon-Th', 'Su 10:30-23:00'], ['Fri-Sa 09:00-01:00'], ['Mon-Fri 10:00-19:45'], ['Mon-Fri 09:00-21:00'], ['Mon-Th 11:30-16:00', '17:00-22:30'], ['Wed-Th 17:00-23:00'], ['Mon-Fri 

In [12]:

# Remove public holidays and summer holidays
# All have PH/SH in them
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs[::-1]):
        if re.search("PH|SH", hr):
            del hours[i][-j-1]

In [13]:
# Functions for determining if statements contain days
day_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)(-(Mon|Tue|Wed|Th|Fri|Sa|Su))?"
time_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))?|off|closed"
time_range_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))|off|closed"
def contains_days(string):
    if re.search(day_re, string):
        return True
    return False

def contains_times(string):
    if re.search(time_re, string):
        return True
    return False



In [14]:
for i,hrs in enumerate(hours):
    # If a time after a comma is missing the days, then we must give it the days from the comma before 
    # e.g. Mon-Th 11:00-12:00, 13:00-15:00 =>  Mon-Th 11:00-12:00, Mon-Th 13:00-15:00
    prev_days = None
    for j, hour in enumerate(hrs):
        if contains_days(hour):
            prev_days = re.search(day_re, hour).group()
        elif prev_days:
            print("HERE", hours[i], prev_days)
            hours[i][j] = prev_days + " " + hours[i][j]
            print("HERE", hours[i], prev_days)

    
    # # If a day before a comma is missing the hours, then we must give it the days from the comma after 
    # # e.g. Mon-Th,Fri 13:00-15:00 =>  Mon-Th 13:00-15:00, Fri 13:00-15:00
    prev_hours = None
    for j, hour in enumerate(hrs[::-1]):
        if contains_times(hour):
            prev_hours = re.search(time_re, hour).group()
        elif prev_hours:
            print("HERE", hrs, prev_hours)
            hours[i][-j-1] = hours[i][-j-1] + " " + prev_hours
            print("HERE", hrs, prev_hours)


HERE ['Mon-Th 11:30-15:30', '17:00-22:30'] Mon-Th
HERE ['Mon-Th 11:30-15:30', 'Mon-Th 17:00-22:30'] Mon-Th
HERE ['Wed', 'Th 16:00-21:00'] 16:00-21:00
HERE ['Wed 16:00-21:00', 'Th 16:00-21:00'] 16:00-21:00
HERE ['Fri', 'Sa 11:00-00:30'] 11:00-00:30
HERE ['Fri 11:00-00:30', 'Sa 11:00-00:30'] 11:00-00:30
HERE ['Sa', 'Su 13:00-04:00'] 13:00-04:00
HERE ['Sa 13:00-04:00', 'Su 13:00-04:00'] 13:00-04:00
HERE ['Fri', 'Sa 11:00-22:00 "Food Hall"'] 11:00-22:00
HERE ['Fri 11:00-22:00', 'Sa 11:00-22:00 "Food Hall"'] 11:00-22:00
HERE ['Mon-Th', 'Su 11:30-24:00'] 11:30-24:00
HERE ['Mon-Th 11:30-24:00', 'Su 11:30-24:00'] 11:30-24:00
HERE ['Tue-Th 11:30-15:00', '17:00-22:00'] Tue-Th
HERE ['Tue-Th 11:30-15:00', 'Tue-Th 17:00-22:00'] Tue-Th
HERE ['Mon-Wed', 'Su 11:30-00:00'] 11:30-00:00
HERE ['Mon-Wed 11:30-00:00', 'Su 11:30-00:00'] 11:30-00:00
HERE ['Su 11:30-15:00', '17:30-20:30'] Su
HERE ['Su 11:30-15:00', 'Su 17:30-20:30'] Su
HERE ['Mon-Th', 'Su 10:30-23:00'] 10:30-23:00
HERE ['Mon-Th 10:30-23:00', '

In [15]:
# Now convert the closing time to a reasonable time if >23:57
# e.g. 11:00-24:00 => 11:00-00:00, 11:00-28:00 => 11:00-04:00
overtime_re = r"([01]\d|2[0-9]):([0-5]\d)(-(2[4-9]):([0-5]\d))"
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        # Find overtime hours
        search = re.search(overtime_re, hr)
        if re.search(overtime_re, hr):
            before = hr[:search.start()+6]
            # Subtract 24 from the time that's >24
            hour_reduced = int(hr[search.start() + 6:search.start()+ 8]) - 24
            after = hr[search.start()+8:]
            # Replace string with new formatted string
            hours[i][j] = f"{before}0{hour_reduced}{after}"
            

In [18]:
# Replace + symbols with -00:00 in the case where it's not already a time range 
# i.e. 11:00+ => 11:00-00:00
# But 11:00-23:00+ => 11:00-23:00
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        if "+" not in hr:
            continue
        
        replace_str = "-00:00"
        if re.search(time_range_re, hr):
            replace_str = ""

        hours[i][j] = hours[i][j].replace("+", replace_str)

In [19]:
hours

[['Wed-Th 16:00-23:00'],
 ['Mon-Th 11:30-15:30', 'Mon-Th 17:00-22:30'],
 ['Mon-Fri 07:00-16:00'],
 ['Wed 06:30-00:00'],
 ['Su 16:00-02:00'],
 ['Sa-Su 11:00-12:00'],
 ['Mon-Su 11:45-23:00'],
 ['Wed 16:00-21:00', 'Th 16:00-21:00'],
 ['Fri 11:00-00:30', 'Sa 11:00-00:30'],
 ['Sa 13:00-04:00', 'Su 13:00-04:00'],
 ['Sa-Su 15:00-21:00'],
 ['Fri 11:00-22:00', 'Sa 11:00-22:00 "Food Hall"'],
 ['Fri-Sa 11:00-01:45'],
 ['Mon-Sa 10:00-00:00'],
 ['Mon 08:00-18:00'],
 ['Mon-Fri 08:00-22:00'],
 ['Tue-Th 11:00-20:00'],
 ['Mon-Th 11:30-00:00', 'Su 11:30-00:00'],
 ['Wed 06:30-16:00'],
 ['Mon-Wed 10:30-00:00'],
 ['Tue-Th 11:30-15:00', 'Tue-Th 17:00-22:00'],
 ['Tue-Fri 07:00-18:00'],
 ['Su 12:00-21:00 "Bar" || Mon-Th', 'Su 11:00-21:00'],
 ['Mon-Wed 11:30-00:00', 'Su 11:30-00:00'],
 ['Su 11:30-15:00', 'Su 17:30-20:30'],
 ['Mon-Th 06:15-19:00'],
 ['04:00-22:00'],
 ['Mon 17:00-01:00'],
 ['Sa-Su 09:30-20:30'],
 ['Su 07:00-19:00'],
 ['Fri 11:00-17:30'],
 ['Mon-Th 10:30-23:00', 'Su 10:30-23:00'],
 ['Fri-Sa 09:00

In [20]:
# Remove days off from opening hours
def remove_days_off(day_str, opening_hours):
    pass

In [21]:
exception_count = 0
for hrs in hours:
    for hr in hrs:
        if len(hr)>=3 and hr.endswith("off"):
            continue
        if len(hr)>=6 and hr.endswith("closed"):
            continue

        try:
            a = OpeningHours.parse(hr, assume_type="24H").json()
        except:
            exception_count += 1
            print(hrs)

['Apr-Dec: Mon-Th 09:00-21:00']
['Apr-Dec: Th-Su 09:00-22:00']
['"Opening 2023"']
['Mon-Su unknown']
['Wed 12:00', 'Th 12:00-1:00']
['Mon-Fri unknown']
['Mar-Nov: Fri 07:00-14:00']
['2024 May 16-23: 12:00-00:00']
['"seasonal"']
['24/7']
['Sep-Jun: Mon-Fri 14:00-19:30']
['Dec-Feb: Fri 07:00-13:00']
['Mon-Su']
['Mon 16:00-04:00', 'Fri 16:00-04:00', 'Sa 28:00', 'Su 7:30-28:00']
['week 1-20 08:30-21:00', '35-52 Mon-Th 08:30-21:00']
['Su 13:00-17:00||"Tue-Wed by appointment', 'Su check calendar online as schedule varies"']
['Mon-Su']
['Mar-Nov Tue 15:00-20:30']
['Mar-Nov Su 12:00-20:30']
['Mon-Th', 'Su']
['Mon-Th', 'Su']
['Mar-Nov Wed 15:00-21:00', 'Th 15:00-21:00']
['Su "by appointment"']
['2024 May 27-30: 12:00-00:00']
['Jan 1 07:00-22:30']
['Mar-Nov Sa 12:00-21:30']
['Under construction']
['Th-Su']
['nan']
['"Temporarily closed"']
['Mar-Nov Fri 15:00-21:30']
['Fri 12:00', 'Sa 12:00-2:00']
['Dec 24 06:00-20:00']
['Sa-Su unknown']
['Dec 31 06:00-20:00']
['Mon-Su 24:00-02:00']
['Jul-Aug: Mo

In [22]:
for hrs in hours:
    for hr in hrs:
        if re.search("PH", hr):
            print(hrs)

In [24]:
exception_count

39